## 🌟动手实践function call

接下来，我们将使用Yi-1.5-9B-Chat模型来实现一个独立的function call示例。这个示例不依赖于任何特定的框架，而是直接使用Hugging Face的transformers库来加载和使用模型。

首先，让我们安装必要的依赖：
⚠️这里请注意你的电脑显存

In [ ]:
!pip install transformers torch

接下来我们开始逐步构建我们的function call实现。
和上面一样我们也使用加减乘三个函数来做示例
#### 步骤1：导入必要的库和定义函数

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 定义可用的函数
def multiply(a: int, b: int) -> int:
    return a * b

def plus(a: int, b: int) -> int:
    return a + b

def minus(a: int, b: int) -> int:
    return a - b
# 在这里你可以添加自己需要的函数
# 函数映射
available_functions = {
    "multiply": multiply,
    "plus": plus,
    "minus": minus
}

#### 步骤2：加载Yi模型和分词器(这一步我们使用transformers进行加载)

In [ ]:
# 加载Yi模型和分词器
model_path = "01-ai/Yi-1.5-9B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

#### 步骤3：实现生成响应和解析函数调用的功能

In [ ]:
# 生成Yi的回复
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, temperature=0.7, top_p=0.95)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Human:")[0].strip()

# 解析输出，提取函数信息
def parse_function_call(response):
    try:
        # 尝试从回复中提取JSON格式的函数调用
        start = response.index("{")
        end = response.rindex("}") + 1
        function_call_json = response[start:end]
        function_call = json.loads(function_call_json)
        return function_call
    except (ValueError, json.JSONDecodeError):
        return None

# 执行函数调用(函数信息传递的桥梁)
def execute_function(function_name: str, arguments: dict) -> Any:
    if function_name in available_functions:
        return available_functions[function_name](**arguments)
    else:
        raise ValueError(f"Function {function_name} not found")

#### 步骤4：实现主循环

In [ ]:
# 主循环
def main():
    # 这个提示词模版中有函数的信息，如果你需要加其它的函数，这里也要和大模型同步
    system_prompt = """You are an AI assistant capable of calling functions to perform tasks. When a user asks a question that requires calling a function, respond with a JSON object containing the function name and arguments. Available functions are:
    - multiply(a: int, b: int) -> int: Multiplies two integers
    - plus(a: int, b: int) -> int: Adds two integers
    - minus(a: int, b: int) -> int: Subtracts two integers
    For example, if the user asks "What is 5 plus 3?", respond with:
    {"function": "plus", "arguments": {"a": 5, "b": 3}}
    If no function call is needed, respond normally."""

    conversation_history = [f"System: {system_prompt}"]

    while True:
        user_input = input("Human: ")
        if user_input.lower() == 'exit':
            break

        # 添加用户输入到对话历史
        conversation_history.append(f"Human: {user_input}")
        
        # 构建完整的提示
        full_prompt = "\n".join(conversation_history) + "\nAssistant:"

        # 获取模型响应
        response = generate_response(full_prompt)
        print(f"Model response: {response}")

        # 解析可能的函数调用
        function_call = parse_function_call(response)

        if function_call:
            function_name = function_call["function"]
            arguments = function_call["arguments"]

            try:
                # 执行函数
                result = execute_function(function_name, arguments)

                # 将结果添加到对话历史
                conversation_history.append(f"Assistant: The result of {function_name}({arguments}) is {result}")
                print(f"Assistant: The result of {function_name}({arguments}) is {result}")
            except Exception as e:
                error_message = f"An error occurred: {str(e)}"
                conversation_history.append(f"Assistant: {error_message}")
                print(f"Assistant: {error_message}")
        else:
            # 如果没有函数调用，直接将模型的响应添加到对话历史
            conversation_history.append(f"Assistant: {response}")
            print(f"Assistant: {response}")

if __name__ == "__main__":
    main()

#### 运行示例

要运行这个示例，你需要确保已经下载了Yi-1.5-9B-Chat模型，或者将`model_path`更改为模型的实际路径。然后，你可以直接运行这个Python脚本。

使用示例：

In [ ]:
Human: What is 5 plus 3?
Model response: Here's the function call to perform the addition:
{"function": "plus", "arguments": {"a": 5, "b": 3}}
Assistant: The result of plus({'a': 5, 'b': 3}) is 8